In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import mplcursors
from random import randrange
import nglview as nv
from nglview.viewer_control import ViewerControl
import os
from urllib.parse import parse_qs,urlparse
import IPython.display
from IPython.core.display import HTML
from ipywidgets import HBox, Layout,VBox
from ipywidgets  import interactive_output

In [2]:
from tools.load_molecule_data import load_data_file
from tools.helper_functions import calc_scaling,unit_rot
from tools.calc_single import single_polar_IR,single_polar_R
from tools.plotting_functions import polar_plot

In [3]:
try:
    query = os.environ.get('QUERY_STRING', '')
    parameters = parse_qs(query)
    molcode=parameters.get('Molcode')[0]
    SH=int(parameters.get('SH')[0])
except:
    molcode="000-823-384"
    SH=0
if SH:
    molfilename="data_SH/"+molcode+".mol"  
    filename="data_SH/"+molcode+".dat"  
else:
    molfilename="data_SAu/"+molcode+".mol"  
    filename="data_SAu/"+molcode+".dat"  

In [4]:
server = os.environ.get('SERVER_NAME', 'localhost') 
url = "http://" + server

port = os.environ.get('SERVER_PORT', '')
if port:
    url += ":" + port
path = os.environ.get('SCRIPT_NAME', '')
if path:
    url += path
else:
    url="http://localhost:8888/voila/render/Documents/Work/Work_website/molecular-vibration-explorer/modes.ipynb"
url="/".join(url.split("/")[:-1])

index_url="<a href="+ url+ "/index.ipynb>Go back to Index</a>"
display(HTML(index_url))
database_url="<a href="+ url+ "/database.ipynb?SH={}".format(SH)+">Go back to database</a>"
display(HTML(database_url))
mol_url="<a href="+ url+ "/molecule.ipynb" +"?Molcode="+molcode+"&SH={}".format(SH)+">Go to molecule page</a>"
display(HTML(mol_url))



In [5]:
fr,Q,D,P,nat,aniso=load_data_file(filename)
nmodes=len(fr)

In [6]:
# read in coordinates from mol file for displaying vibrations
def read_mol(molfilename):
    molcoords=[]
    with open(molfilename,'r') as inpfile:
        line=inpfile.readline()
        while line:
            spl=line.split()
            if len(spl)==16:
                molcoords.append([float(spl[0]),float(spl[1]),float(spl[2])])
            line=inpfile.readline()
    return molcoords

molcoords=read_mol(molfilename)

In [7]:
%matplotlib widget
# create polar plot
cm_to_thz=0.02998
pi=math.pi
torad=2*pi/360

def plot_displacements(mode,sc):
    modenum=int(mode.split(":",1)[0])
    m=modenum-1 # Python starts from 0
    display(widgets.Label(value="Molecule "+molcode,layout = widgets.Layout(display="flex", 
                                                                            justify_content="center",width='450px')))
   # display(widgets.Label(value=molfilename.split("/")[-1]))
    #print(molfilename)
    # Display molecule and chosen vibration
    view = nv.NGLWidget()
    comp = view.add_component(molfilename) 
    
    control = ViewerControl(view=view)
  #  control.spin([1,0,0],-90*torad)
  #  control.spin([0,0,1],40*torad)

    for at in range(nat):
        displ1=list(molcoords[at]-sc*Q[m,at])
        displ2=list(molcoords[at]+sc*Q[m,at])
        view.shape.add_arrow(displ1, displ2, [ 0, 0, 0 ], 0.2)
    # show axes
    view.shape.add_arrow([-4,-4,-4], [-2,-4,-4], [ 0, 0, 1 ], 0.2, 'x')
    view.shape.add_arrow([-4,-4,-4], [-4,-2,-4], [ 1, 0, 0 ], 0.2, 'y')
    view.shape.add_arrow([-4,-4,-4], [-4,-4,-2], [ 0, 0.8, 0.2 ], 0.2, 'z')
    view.shape.add('text', [-1.5,-4,-4], [ 0, 0, 1 ], 3, 'x')
    view.shape.add('text', [-4, -1.5, -4], [ 1, 0, 0 ], 3, 'y')
    view.shape.add('text', [-4, -4, -1.5], [ 0, 0.8, 0.2 ], 3, 'z')
 #   display(view)
    view.control.zoom(0.3)
    from ipywidgets import Box


    box = Box([view])
    box.layout.width = '450px'
    box.layout.height = '420px'
    display(box)
    
    layout=widgets.Layout(width='120px')
    labellayout=widgets.Layout(width='120px')
    
    frequencycm=HBox([widgets.Label(value=r'Frequency /cm$^{-1}$:',layout=labellayout),widgets.Text(value="{:.1f}".format(fr[m]),
                                                            step=0.1,disabled=True,layout=layout)])
    frequencyTHz=HBox([widgets.Label(value=r'Frequency /THz:',layout=labellayout),widgets.Text(value="{:.3f}".format(cm_to_thz*fr[m]),
                                                            step=0.1,disabled=True,layout=layout)])
    display(frequencycm,frequencyTHz)

def plot_single_mode(mode,rb_type_study,rb_polar_plan,rb_ir_beam,rb_rin_beam,rb_rout_beam):
    modenum=int(mode.split(":",1)[0])
    m=modenum-1 
    polarizations=[np.array([1.,0.,0.]),np.array([0.,1.,0.]),np.array([0.,0.,1.])]
    theta = np.linspace(0, 2*pi, num=200, endpoint=True) 
    nb_theta=len(theta)
    I=np.zeros(nb_theta)
    proj=rb_polar_plan
    e=polarizations[rb_ir_beam]
    e_in=polarizations[rb_rin_beam]
    e_out=polarizations[rb_rout_beam]
    
    for t in range (0,nb_theta) :
        r=unit_rot(proj,theta[t])
        if rb_type_study==1 :
            I[t]=single_polar_IR(D[m,:],r,e)
        elif rb_type_study==2 :
            I[t]=single_polar_R(P[m,:,:],r,e_in,e_out)
        elif rb_type_study==3 :
            I[t]=single_polar_IR(D[m,:],r,e)*single_polar_R(P[m,:,:],r,e_in,e_out)
    
    I=I/np.max(I)
    line=polar_plot(theta,I)
    c1 = mplcursors.cursor(line)
    @c1.connect("add")
    def _(sel):
        sel.annotation.get_bbox_patch().set(fc="white")
        sel.annotation.arrow_patch.set(arrowstyle="simple", fc="white", alpha=.5)

    ax=plt.gca()
    plt.tight_layout()
    plt.show()
    display(widgets.Label(value="Click to create cursor, Right-click to remove it"))

### Normal modes

In [8]:
layout=widgets.Layout(width='280px', description_width='200px')
style = {'description_width': '120px'}

axes=[["x",0], ["y",1] ,["z",2]]
studies=[["IR absorption",1], ["Raman scattering",2] ,["Conversion/SFG",3]]
planes=[["x-y plane",1], ["x-z plane",2] ,["y-z plane",3]]

options=[]
for n,f in enumerate(fr):
    options.append("{}: {:.1f} cm-1".format(n+1,f))

mode=widgets.Dropdown(options=options, #np.arange(1,nmodes+1),
    value=options[0],
    description='Mode:',layout=layout,style=style
)

sc=widgets.FloatSlider(value=3.5,min=1,max=10,step=0.1,description='Scale arrows:',
                                layout=layout,
                                  style=style)

rb_type_study=widgets.RadioButtons(
                        options=studies,
                        value=1,
                        description='Spectroscopy:',
                        layout=layout,
                        style=style,
                        disabled=False
                    )
rb_polar_plan=widgets.RadioButtons(
                        options=planes,
                        value=2,
                        description='Projection:',
                        layout=layout,
                        style=style,
                        disabled=False
                    )
rb_ir_beam=widgets.RadioButtons(
                        options=axes,
                        value=2,
                        description='THz/IR beam:',
                        layout=layout,
                        style=style
                    )
rb_rin_beam=widgets.RadioButtons(
                        options=axes,
                        value=2,
                        description=r'Raman $\it{in}$ beam:',
                        layout=layout,
                        style=style
                    )
rb_rout_beam=widgets.RadioButtons(
                        options=axes,
                        value=2,
                        description=r'Raman $\it{out}$ beam:',
                        layout=layout,
                        style=style
                    )
     
int_mode_plot = interactive_output(plot_displacements, {'mode':mode, 'sc':sc})

int_polar_plot = interactive_output(plot_single_mode,  
         {'mode':mode,
         'rb_type_study':rb_type_study,
         'rb_polar_plan':rb_polar_plan,
         'rb_ir_beam':rb_ir_beam,
         'rb_rin_beam':rb_rin_beam,
         'rb_rout_beam':rb_rout_beam
         })

controls =VBox([mode,sc,rb_type_study,rb_polar_plan,rb_ir_beam,rb_rin_beam,rb_rout_beam]) 

controls.layout.height = '600px'
controls.layout.width = '300px'
display(HBox([controls,int_mode_plot,int_polar_plot], layout=Layout(justify_content='space-around')))